In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

### 하이퍼파라미터
* 하이퍼파라미터는 모델 최적화 프로세스를 제어할 수 있는 조정 가능한 파라미터입니다.
- 에포크 수 : 데이터셋을 반복할 횟수입니다.
- 배치 크기 : 파라미터가 업데이트되기 전에 네트워크를 통해 전달되는 데이터 샘플의 수입니다.
- 학습율 : 각 배치/에포크에서 모델 파라미터를 업데이트할 양입니다. 값이 작을수록 학습 속도가 느려지고, 값이 클수록 학습 중에 예측할 수 없는 동작이 발생할 수 있습니다.

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

### 최적화(Optimization) 루프
* 최적화 루프의 각 반복을 에포크라고 합니다.
- 학습 루프 : 학습 데이터셋을 반복하여 최적의 파라미터로 수렴을 시도합니다.
- 검증/테스트 루프 : 테스트 데이터셋을 반복하여 모델 성능이 개선되고 있는지 확인합니다.

### 손실 함수
* 손실 함수는 얻은 결과와 목표 값의 차이 정도인 오차를 측정하는 것
* 학습 과정에서 최소화하고자 하는 것이 바로 손실 함수
* 오차를 계산하기 위해 주어진 데이터 샘플의 입력을 사용하여 예측을 하고 이를 실제 데이터 레이블 값과 비교

In [3]:
loss_fn = nn.CrossEntropyLoss()

### Optimizer
* 최적화는 각 학습 단계에서 모델 오류를 줄이기 위해 모델 파라미터를 조정하는 프로세스
* 최적화 알고리즘은 이 프로세스가 수행되는 방식을 정의

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### 전체 구현
* 최적화 코드를 반복하는 train_loop와 테스트 데이터에 대해 모델의 성능을 평가하는 test_loop를 정의

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # 모델을 학습 모드로 설정 - 일괄 정규화 및 드롭아웃 레이어에 중요함
    # 이 상황에서는 불필요하지만 모범 사례를 위해 추가되었습니다.
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # 예측 및 손실 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # 모델을 평가 모드로 설정 - 배치 정규화 및 드롭아웃 레이어에 중요합니다.
    # 이 상황에서는 불필요하지만 모범 사례를 위해 추가되었습니다.
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # torch.no_grad()로 모델을 평가하면 테스트 모드에서 그라데이션이 계산되지 않습니다.
    # 또한 requires_grad = True인 텐서에서 불필요한 그라데이션 계산과 메모리 사용량을 줄이는 역할도 합니다.
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.294755  [   64/60000]
loss: 2.289910  [ 6464/60000]
loss: 2.277708  [12864/60000]
loss: 2.271257  [19264/60000]
loss: 2.257020  [25664/60000]
loss: 2.217723  [32064/60000]
loss: 2.230270  [38464/60000]
loss: 2.194645  [44864/60000]
loss: 2.192195  [51264/60000]
loss: 2.161500  [57664/60000]
Test Error: 
 Accuracy: 47.6%, Avg loss: 2.159667 

Epoch 2
-------------------------------
loss: 2.169229  [   64/60000]
loss: 2.158765  [ 6464/60000]
loss: 2.110713  [12864/60000]
loss: 2.116802  [19264/60000]
loss: 2.072376  [25664/60000]
loss: 2.011608  [32064/60000]
loss: 2.037680  [38464/60000]
loss: 1.966541  [44864/60000]
loss: 1.964715  [51264/60000]
loss: 1.892347  [57664/60000]
Test Error: 
 Accuracy: 58.5%, Avg loss: 1.895185 

Epoch 3
-------------------------------
loss: 1.937343  [   64/60000]
loss: 1.899254  [ 6464/60000]
loss: 1.795655  [12864/60000]
loss: 1.813076  [19264/60000]
loss: 1.709141  [25664/60000]
loss: 1.667702  [32064/600